 # Opis notatnika
 Ten notatnik jest kontunacją analizy danych o lotach i ich opóźnieniach. Od tego momentu zaczniemy łączyć posiadana przez nas zbiory danych, będąc w stanie dokonać dodatkowych analiz.

 Zanim jednak do tego przejdziemy, należy, podobnie jak w poprzednim kroku, skonfigurować odpowiednio notatnik.
 
 W tej części warsztatu ponownie wcielasz się w rolę Analiyka Danych, którego zadaniem jest wykonanie analizy eksplotacyjnej zbioru danych - jedno z wymagań dostarczonych przez klienta.

 Tutaj zaimportuj wymagane biblioteki

In [ ]:
import mysql.connector as sql
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
import plotly.express as px

 ## Połączenie z bazą danych
 Tutaj uzupełnij konfigurację połączenia

In [ ]:
username = 'postgres'
password = 'postgres'

host = 'localhost'
database = 'airlines'
port = 5432

 Tutaj stwórz zmienną engine, która zostanie użyta do połączenia z bazą danych

In [ ]:
url = f'postgresql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(url)

 Tutaj uzupełnij implementację metody `read_sql_table`

In [ ]:
def read_sql_table(table_name):
    try:
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, con=engine)
        print(f"Data from table '{table_name}' loaded successfully.")
        return df
    except Exception as e:
        print(f"Failed to load data from table '{table_name}': {e}")
        return None

 Tutaj zaczytaj zapisaną wcześniej ramkę danych `flight_df` do zmniennej o takiej samej nazwie

In [ ]:
flight_df = pd.read_csv('flight_df_01.csv')

Sprawdzenie poprawności danych w ramce `flight_df` 

In [ ]:
flight_df_expected_rows_amount = 1057391
flight_df_rows_amount = flight_df.shape[0]

assert flight_df_rows_amount == flight_df_expected_rows_amount, f'Oczekiwano {flight_df_expected_rows_amount} wierszy, otrzymano {flight_df_rows_amount}'

 # Wzbogacenie o `aircraft`
 Używając procedury `read_sql_table` wczytaj dane z tabeli `aircraft` i zapisz jako `aircraft_df`. Następnie:  
 1. Usuń z ramki kolumny `number_of_seats` oraz `id`. Na tej podstawie usuń nadmiarowe wiersze (duplikaty).  
 1. Następnie jeszcze raz sprawdź, czy dla kolumny `tail_num` nie występują duplikaty. Innymi słowy należy sprawdzić, czy dla jednego `tail_num` występuje więcej niż jeden rok produkcji.  
 1. Jeśli tak to:  
     - do ramki `aircraft_df_duplicated` zapisz powielone zgodnie ze sprawdzeniem wiersze,  
     - zgodnie z powyższym zmodyfikuj ramkę tak, aby w przypadku duplikatu za datę wytworzenia samolotu, uznana została najnowsza tj. jeśli dla `tail_num` są dostępne daty produkcji 1998 oraz 2001, uznajemy, że `tail_num` został wyprodukowany w `2001`.

 Wskazówki:
 - Praca z duplikatami na LMS: `Python - analiza danych -> Dzień 5 - Pandas -> Duplikaty`
 - Dokumentacja metody `duplicated`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html)
 - Dokumentacja metody `drop_duplicates`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

 Tutaj wczytaj tabelę `aircraft` używając `read_sql_table`

In [ ]:
aircraft_df = pd.read_sql_table('aircraft', con=engine)

 Tutaj usuń kolumny `number_of_seats`, `id` oraz duplikaty z ramki `aircraft_df`

In [ ]:
aircraft_df = aircraft_df.drop(columns=['number_of_seats', 'id'])

In [ ]:
aircraft_df = aircraft_df.drop_duplicates()

In [ ]:
aircraft_df['tail_num'].duplicated().sum()

In [ ]:
aircraft_df_duplicated = aircraft_df[aircraft_df['tail_num'].duplicated(keep=False)].copy()

In [ ]:
aircraft_df_duplicated

In [ ]:
aircraft_df_duplicated = aircraft_df_duplicated.groupby('tail_num').apply(lambda x: x.loc[x['manufacture_year'].idxmax()]).reset_index(drop=True)

In [ ]:
aircraft_df_duplicated

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
aircraft_df_expected_rows = 7364
aircraft_df_expected_columns = set(['tail_num', 'manufacture_year'])

aircraft_df_rows = aircraft_df.shape[0]

diff = aircraft_df_expected_columns.symmetric_difference(set(aircraft_df.columns))
assert aircraft_df_rows == aircraft_df_expected_rows, f'Spodziewano się {aircraft_df_expected_rows} wierszy , otrzymano {aircraft_df_rows} wierszy'

assert diff == set([]), f'Spodziewano się {aircraft_df_expected_columns} kolumn, otrzymano: {aircraft_df_expected_columns} kolumn. Różnica: \n\t{diff}'

 Tutaj sprawdź czy w ramkce `aircraft_df` występują duplikaty wewnątrz kolumny `tail_num`. Czyli czy dla danego `tail_num` występuje więcej niż jeden rok produkcji.

In [ ]:
aircraft_df_is_duplicated = aircraft_df.duplicated(subset='tail_num')
aircraft_df_duplicated = aircraft_df.loc[aircraft_df_is_duplicated]

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić czy ta część została poprawnie wykonana

In [ ]:
aircraft_df_expected_rows = 3
aircraft_df_duplicated_rows = aircraft_df_duplicated.shape[0]
assert aircraft_df_duplicated_rows == aircraft_df_expected_rows, f"Oczekiwano {aircraft_df_expected_rows} wierszy, otrzymano {aircraft_df_duplicated_rows}"

 ## Modyfikacja `aircraft_df`
 Tutaj dokonaj aktualizacji tabeli `aircraft_df` - jeśli jest taka potrzeba. Zrób to tak aby, dla powielonych `tail_num`, `manufacture_year` został ustawiony jako najwyższy

In [ ]:
aircraft_df['manufacture_year'] = aircraft_df.groupby('tail_num')['manufacture_year'].transform('max')

In [ ]:
aircraft_df = aircraft_df.drop_duplicates(subset='tail_num')

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
test_tail = 'N783CA'
test_value = aircraft_df.loc[aircraft_df['tail_num']
                             == test_tail]['manufacture_year']
test_value = int(test_value)

expected_value = 2000
assert test_value == expected_value, f"Dla 'tail_num' == '{test_tail}' oczekiwano {expected_value} otrzymano {test_value}"

In [ ]:
test_tail = 'N783CA'
test_value = aircraft_df.loc[aircraft_df['tail_num'] 
                             == test_tail, 'manufacture_year'].iloc[0]
test_value = int(test_value)

expected_value = 2000
assert test_value == expected_value, f"Dla 'tail_num' == '{test_tail}' oczekiwano {expected_value} otrzymano {test_value}"

 ## Połączenie `aircraft_df` oraz `flight_df`
 Połącz ramkę `aircraft_df` oraz `flight_df` zgodnie z kluczem oraz odpowiedz na następujące pytania:
 1. Czy po połączeniu pojawiły się duplikaty? Dokonaj odpowiedniego sprawdzenia.
 1. Wyznacz zależność roku produkcji do częstotliwości opóźnień. Wynik zapisz do tabeli `delays_by_manufacture_year_df`.
 1. Przedstaw wyniki w tabeli za pomocą wykresu punktowego.
 1. Dokonaj modyfikacji w taki sposób, aby wyświetlone na wykresie zostały tylko takie roczniki samolotów, które wykonały łącznie więcej niż 10000 `(>)` lotów.

> **Wskazówka:**
> Aby nie utracić potencjalnie całej dotychczasowej pracy, zapisz wynik do tymczasowej zmiennej np. `tmp_flight_df`. Po sprawdzeniu możesz użyć metody `copy`: [link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html), aby nadpisać `flight_df` używając `tmp_flight_df`.

 Tutaj dokonaj złączenia `flight_df` z `aircraft_df`, wynik zapisz do `tmp_flight_df`

In [ ]:
tmp_flight_df = flight_df.merge(aircraft_df, on='tail_num', how='left')

 Tutaj dokonaj sprawdzenia, czy nie pojawiły się duplikaty

In [ ]:
duplicates = tmp_flight_df.duplicated(subset=['id'], keep=False)

In [ ]:
duplicates_number = duplicates.sum()
duplicates_number

 Tutaj nadpisz `flight_df` używając `tmp_flight_df`

In [ ]:
flight_df = tmp_flight_df.copy(deep=True)

 ## Opóźnienia vs. rok produkcji samolotu cz. 1
 Tutaj wyznacz zależność roku produkcji do częstotliwości opóźnień

In [ ]:
delays_by_manufacture_year_df = tmp_flight_df.groupby('manufacture_year').agg(
    total_flights=('id', 'count'),
    delayed_flights=('is_delayed', 'sum')
)

delays_by_manufacture_year_df['delayed_percentage'] = (delays_by_manufacture_year_df['delayed_flights'] / delays_by_manufacture_year_df['total_flights']) * 100

In [ ]:
delays_by_manufacture_year_df = delays_by_manufacture_year_df.reset_index()
delays_by_manufacture_year_df.head()

 Tutaj wyrysuj ramkę `delays_by_manufacture_year_df`

In [ ]:
# Tworzenie wykresu punktowego
fig1 = px.scatter(
    delays_by_manufacture_year_df,
    x='manufacture_year',
    y='delayed_percentage',
    title='Manufacture year vs Percentage of delayed flights',
    labels={'manufacture_year': 'Manufacture year', 'delayed_percentage': 'Percentage of delayed flights'},
    opacity=0.5
)

# Wyświetl wykres
fig1.show()

 Tutaj zmodyfikuj wykres tak, aby prezentował tylko te roczniki, które odbyły więcej niż 10000 lotów

In [ ]:
delays_by_manufacture_year_10000_df = delays_by_manufacture_year_df[delays_by_manufacture_year_df['total_flights'] > 10000]
delays_by_manufacture_year_10000_df.head()

In [ ]:
# Tworzenie wykresu punktowego
fig2 = px.scatter(
    delays_by_manufacture_year_10000_df,
    x='manufacture_year',
    y='delayed_percentage',
    title='Manufacture year vs Percentage of delayed flights',
    labels={'manufacture_year': 'Manufacture year', 'delayed_percentage': 'Percentage of delayed flights'},
    opacity=0.5
)

# Wyświetl wykres
fig2.show()

 ## Opóźnienia vs. rok produkcji samolotu cz. 2
 Dokonaj agregacji kolumny `manufacture_year` do kolumny `manufacture_year_agg` zgodnie z poniższym:
 1. Grupując dane co 3 lata -> Czy po grupowaniu można zauważyć zależność? Wyniki zapisz do ramki `flight_delays_by_manufacture_year_agg_df`.
 1. Wyznacz top 5 roczników samolotu, które wykonały najwięcej lotów. Wyniki zapisz do ramki `top_manufactured_df`, do obliczeń wykorzystaj `delays_by_manufacture_year_df`.

 Tutaj dodaj kolumnę `manufacture_year_agg` do ramki `flight_df`

In [ ]:
earliest_manufacture_year = flight_df['manufacture_year'].min()
earliest_manufacture_year

In [ ]:
bins = range(int(earliest_manufacture_year), int(flight_df['manufacture_year'].max()) + 3, 3)
labels = [f'{i}-{i+3}' for i in bins[:-1]]

flight_df.loc[:, 'manufacture_year_agg'] = pd.cut(flight_df['manufacture_year'], bins=bins, labels=labels, right=False)

In [ ]:
flight_df.head()

 Tutaj stwórz zmienną `flight_delays_by_manufacture_year_agg_df`

In [ ]:
flight_delays_by_manufacture_year_agg_df = flight_df.groupby('manufacture_year_agg').agg(
    total_flights=('id', 'count'),
    delayed_flights=('is_delayed', 'sum')
).reset_index()

In [ ]:
flight_delays_by_manufacture_year_agg_df['delayed_percentage'] = (flight_delays_by_manufacture_year_agg_df['delayed_flights'] / flight_delays_by_manufacture_year_agg_df['total_flights']) * 100
flight_delays_by_manufacture_year_agg_df

 Tutaj stwórz wykres w oparciu o dane zawarte w `flight_delays_by_manufacture_year_agg_df`

In [ ]:
# Tworzenie wykresu punktowego
fig3 = px.scatter(
    flight_delays_by_manufacture_year_agg_df,
    x='manufacture_year_agg',
    y='delayed_percentage',
    title='Manufacture year range vs Percentage of delayed flights',
    labels={'manufacture_year_agg': 'Manufacture year', 'delayed_percentage': 'Percentage of delayed flights'},
    opacity=0.5
)

# Wyświetl wykres
fig3.show()

Tutaj wyznacz TOP 5 roczników produkcji - czyli sortując według liczby wykonanych lotów, pamiętaj o wyświetleniu również wartości opóźnienia.

In [ ]:
top_manufactured_df = delays_by_manufacture_year_df.sort_values(by='total_flights', ascending=False).head(5)
top_manufactured_df

 # Podsumowanie
 W tym notatniku do naszej wyjściowej ramki danych `flight_df` dołączyliśmy tabelę `aircraft_df` i za jej pomocą dodaliśmy kolejny wymiar do naszej analizy. Zauważmy, ile dodatkowych wniosków mogliśmy wyciągnąć dzięki jej dodaniu.

 Zanim przejdziemy dalej, należy zapisać bieżącą postać ramki (najlepiej lokalnie), która zostanie użyta w kolejnym notatniku.

 > **Wskazówka:**  
 > Aby uniknąć potencjalnych problemów, najlepiej zapisać ramkę z nazwą nawiązującą do tego notatnika, np. `flight_df_01`.

 Tutaj zapisz ramkę w najdogodniejszy sposób

In [ ]:
flight_df.to_csv(r'..\data\processed\flight_df_02.csv', index=False)